In [109]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

nb = MultinomialNB()
svm = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)
rf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)

print score_predictions(nb), ' -  naive bayes'
print score_predictions(svm), ' -  support vector machine'
print score_predictions(rf), ' -  random forest'

0.6875  -  naive bayes
0.78125  -  support vector machine
0.65625  -  random forest


In [107]:
from __future__ import division

def score_predictions(learner):
    import random
    import sqlite3

    sqlite_file = 'abc_code.db'
    table_name = 'fake_scores'

    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()

    column1 = 'question_id'
    question_id = 'question-1'
    column2 = 'score'
    limit = 10

    data = list()
    target = list()

    for points in (0, 1, 2, 3):
        c.execute('SELECT answered, score FROM {tn} WHERE {cn1}="{val1}" AND {cn2}="{val2}" LIMIT {limit}'.\
                format(tn=table_name, cn1=column1, val1=question_id, cn2=column2, val2=points, limit=limit))
        rows = c.fetchall()

        data.extend([x[0] for x in rows])
        target.extend([x[1] for x in rows])

    conn.close()

    import sys
    sys.path.append('/Library/Python/2.7/site-packages')
    from sklearn.feature_extraction.text import CountVectorizer
    # CountVectorizer(analyzer='char_wb', ngram_range=(2, 2), min_df=1)
    from sklearn.pipeline import Pipeline
    from sklearn.feature_extraction.text import TfidfTransformer

    text_clf = Pipeline([('vect', CountVectorizer(analyzer='char_wb', ngram_range=(2, 2), min_df=1)),
                         ('tfidf', TfidfTransformer()),
                         ('clf', learner),
    ])

    text_clf = text_clf.fit(data, target)

    test_data = ['if then', 
                 'yes no', 
                 'instructions depend', 
                 'case depend', 
                 'not related', 
                 'Very often when you write code, you want to perform different actions for different decisions. You can use conditional statements in your code to do this',
                 'Conditional statements are used to perform different actions based on different conditions.',
                 'In computer science, conditional statements, conditional expressions and conditional constructs are features of a programming language, which perform different computations or actions depending on whether a programmer-specified boolean condition evaluates to true or false',
                ]
    test_target = [3, 3, 3, 3, 0, 3, 3, 3]
    predicted = text_clf.predict(test_data)
#     print 'predic: ', [x for x in predicted]
#     print 'target: ', test_target

    def score_prediction(predicted, target, target_names):
        possible = len(target) * len(target_names)
        score = possible
        for p, t in zip(predicted, target):
            diff = abs(t - p)
            score -= diff
        return score / possible

    return score_prediction(predicted, test_target, ["criteria-4", "criteria-3", "criteria-2", "criteria-1"])